In [ ]:
# %load /p/home/jusers/langguth1/jureca/atmorep_github/atmorep/atmorep/applications/datasets/downscaling_data_loader.py
####################################################################################################
#
#  Copyright (C) 2022
#
####################################################################################################
#
#  project     : atmorep
#
#  author      : atmorep collaboration
# 
#  description :
#
#  license     :
#
####################################################################################################

import torch
import numpy as np
import zarr
import pandas as pd
import xarray as xr
from datetime import datetime
import time
import os, sys
from pathlib import Path

In [ ]:
#sys.path.insert(0, "/p/project/deepacf/atmo-rep/langguth1/atmo-rep/atmorep/")
sys.path.insert(0, "/p/project/deepacf/atmo-rep/patnala1/atmorep/")
from atmorep.datasets.normalizer import normalize
from atmorep.utils.utils import tokenize, get_weights
from atmorep.applications.downscaling.utils.era5_imerg_data_aligner import lat_lon_time_range
from atmorep.utils.logger import logger

In [ ]:
def math_to_geolats(lats_math):
    """
    Convert 'mathematical' latitudes as used in AtmoRep to geographical latitudes.
    Example conversions:
        Mathematical latitude: 0° -> Geographical latitude: 90°N
        Mathematical latitude: 180° -> Geographical latitude: -90°N/90°S
    :param lats_math: 1D-array with mathematical latitudes
    :return lats_geo: 1D-array with geographical latitudes
    """
    lats_geo = np.where(lats_math > 90, -lats_math + 90, np.abs(lats_math - 90.))

    return lats_geo

def geo_to_mathlats(lats_geo):
    """
    Convert geographical latitude to 'mathematical' latitudes as used in AtmoRep.
    Example conversions:
        Geographical latitude: 90°N ->  Mathematical latitude: 0°N
        Geographical latitude: -90°N -> Mathematical latitude: 180°
    :param lats_math: 1D-array with geographical latitudes
    :return lats_geo: 1D-array with mathematical latitudes
    """
    lats_math = np.where(lats_geo > 0, np.abs(lats_geo - 90), -lats_geo + 90.)
    return lats_math

def get_latlon_arr(lat_range, lon_range, res):
    lats = np.arange(lat_range[0], lat_range[1] + res/10., res)
    lons = np.arange(lon_range[0], lon_range[1] + res/10., res)

    return lats, lons

In [ ]:
class MultifieldDownscalingSampler( torch.utils.data.IterableDataset):
    
    def __init__(self, input_file_path, target_file_path, input_fields, target_fields, years, batch_size, n_size,
                 num_samples, downscaling_ratio, with_shuffle=False, with_source_idxs = True, with_target_idxs=True) :
        '''
          Iterable torch dataset for ERA5 to IMERG downscaling task loading data at an arbitrary number of vertical levels
    
          input_file_path: path to input ERA5 zarr storage
          target_file_path: path to input IMERG zarr storage
          input_fields: configuration list for input data fields
          target_fields: configuration list for input data fields
          years: year or list of years from which data should be sampled
          batch_size: number of samples per batch
          nsize : neighborhood in (tsteps, nlat_ERA5, nlon_ERA5) -> different to MultifieldDataSampler!!!
          num_samples: total number of samples to draw
          downscaling_ratio: downscaling factor between ERA5 input and IMERG target data
          with_shuffle: flag for shuffling samples
          with_source_idxs:
          with_target_idxs:   
        '''       
        super(MultifieldDownscalingSampler, self).__init__()

        self.input_fields = input_fields
        self.target_fields = target_fields
        self.n_size = n_size
        self.num_samples = num_samples
        self.with_source_idxs = with_source_idxs
        self.with_target_idxs = with_target_idxs
        self.batch_size = batch_size
        self.with_shuffle = with_shuffle
        self.downscaling_ratio = downscaling_ratio

        if not os.path.exists(input_file_path):
            FileNotFoundError(f"Input zarr store {input_file_path} does not exist.")

        if not os.path.exists(input_file_path):
            FileNotFoundError(f"Target zarr store {target_file_path} does not exist.")

        # open zarr stores
        self.era5_ds = zarr.group(input_file_path)
        self.imerg_ds = zarr.group(target_file_path)

        # get coordinate information from ERA5 and IMERG data
        self.era5_res = self.era5_ds.attrs['res']
        self.imerg_res = self.imerg_ds.attrs['res']

        self.era5_times = self.era5_ds["time"]
        self.era5_lats = np.array(self.era5_ds['lats'])
        self.era5_lons = np.array(self.era5_ds['lons'])

        self.imerg_times = self.imerg_ds["time"]
        self.imerg_lats = np.array(self.imerg_ds['lats'])
        self.imerg_lons = np.array(self.imerg_ds['lons'])
        self.imerg_lons = np.where(self.imerg_lons < 0, self.imerg_lons + 360, self.imerg_lons)

        self.era5_num_lats = self.era5_ds['lats'].shape[0]
        self.era5_num_lons = self.era5_ds['lons'].shape[0]

        # spatial shift of first IMERG data point w.r.t. first ERA5 data point
        # Note: IMERG grid must already be centered around ERA5 grid accordingly during preprocessing
        self.dx_shift = (self.downscaling_ratio - 1)/2.*np.array(self.imerg_res)

        # Note: Neighborhood sampling is based on ERA5 indices which serve as anchor points
        # Parameters for centered spatial slicing
        # In case that the number of grid points is even, the center (anchor) point for the local neighborhood is shifted west/south
        # Example nor n_size=4: 
        #  |           |   |           |
        #  x   o   x   x   x   o   x   x 
        #  |           |   |           |
        # The leftmost and rightmost neighborhoods incl. their boundaries | and anchor points o are shown. 
        nsize_lat_half = int((self.n_size[1] - 1 )/2)
        self.nsize_lat = np.asarray([nsize_lat_half, nsize_lat_half])
        if self.n_size[1] % 2 == 0:
            self.nsize_lat[1] += 1

        nsize_lon_half = int((self.n_size[2] - 1 )/2)
        self.nsize_lon = np.asarray([nsize_lon_half, nsize_lon_half])
        if self.n_size[2] % 2 == 0:
            self.nsize_lon[1] += 1

        # ensure that sampling is restricted to spatio-temporal index ranges where ERA5 and IMERG data are available
        _, tidx_era5, _ = np.intersect1d(self.era5_times, self.imerg_times, return_indices=True)
        _, latidx_era5, _ = np.intersect1d(self.era5_lats, self.imerg_lats, return_indices=True)
        _, lonidx_era5, _ = np.intersect1d(self.era5_lons, self.imerg_lons, return_indices=True)
        
        # handle situation where the contigious domain cross the zero meridian
        split_idx = np.where(np.diff(lonidx_era5) != 1)[0] + 1    # find indices where the difference between consecutive elements is not 1
        
        # Add the start and end indices to form ranges
        idx_segs = np.split(lonidx_era5, split_idx)
        contiguous_segs = [(idx_seg[0], idx_seg[-1]) for idx_seg in idx_segs]

        # save ERA5 anchor point indices for sampling
        self.range_ilat = np.array([min(latidx_era5) + self.nsize_lat[0], max(latidx_era5) - self.nsize_lat[1]])
        self.anchor_ilat = np.arange(*self.range_ilat)
        
        print("***Valid Latitude range***")
        print(min(latidx_era5), max(latidx_era5))
        
        if len(contiguous_segs) == 2:
            range_ilon = [min(contiguous_segs[1]), max(contiguous_segs[0])]
        elif len(contiguous_segs) == 1:
            range_ilon = [min(lonidx_era5), max(lonidx_era5)]
        else:
            raise ValueError(f"More than two splitting indices identified along longitude dimension. Ensure that IMERG-domain is contigious.")


        self.range_ilon = np.array([range_ilon[0] + self.nsize_lon[0], range_ilon[1] - self.nsize_lon[1]])
        if self.range_ilon[0] > self.range_ilon[1]:
            self.anchor_ilon = np.arange(self.range_ilon[0], self.range_ilon[1] + self.era5_num_lons)
            self.anchor_ilon = self.anchor_ilon%self.era5_num_lons
        else:
            self.anchor_ilon = np.arange(*self.range_ilon)       

        print("***Valid longitude range***")
        print(range_ilon[0], range_ilon[1])

        # Note: sample time-index is not centered, but placed at the end of the time sequence
        # add n_size[0] to avoid sampling from time steps at whch no IMERG data is available
        self.range_itime = np.array([min(tidx_era5) + self.n_size[0], max(tidx_era5)])
        
        self.year_base = self.era5_ds['time'][self.range_itime[0]].astype(datetime).year

        # get normalizers for input and target data
        self.input_normalizers = []

        for ifield, field_info in enumerate(input_fields):
            corr_type = 'global' if len(field_info) <=6 else field_info[6]
            nf_name = 'global_norm' if corr_type == 'global' else 'norm'
            self.input_normalizers.append([])
            for vl in field_info[2]: 
                if vl == 0:
                    field_idx = self.era5_ds.attrs['fields_sfc'].index( field_info[0])
                    n_name = f'normalization/{nf_name}_sfc'
                    self.input_normalizers[ifield] += [self.era5_ds[n_name].oindex[ :, :, field_idx]] 
                else:
                    vl_idx = self.era5_ds.attrs['levels'].index(vl)
                    field_idx = self.era5_ds.attrs['fields'].index( field_info[0])
                    n_name = f'normalization/{nf_name}'
                    self.input_normalizers[ifield] += [self.era5_ds[n_name].oindex[ :, :, field_idx, vl_idx] ]


        self.target_normalizers = []

        for ifield, field_info in enumerate(target_fields):
            corr_type = 'global' if len(field_info) <=6 else field_info[6]
            nf_name = 'global_norm' if corr_type == 'global' else 'norm'
            self.target_normalizers.append([])
            for vl in field_info[2]: 
                if vl == 0:
                    field_idx = self.imerg_ds.attrs['fields_sfc'].index( field_info[0])
                    n_name = f'normalization/{nf_name}_sfc'
                    self.target_normalizers[ifield] += [self.imerg_ds[n_name].oindex[ :, :, field_idx]] 
                else:
                    vl_idx = self.target_ds.attrs['levels'].index(vl)
                    field_idx = self.imerg_ds.attrs['fields'].index( field_info[0])
                    n_name = f'normalization/{nf_name}'
                    self.target_normalizers[ifield] += [self.imerg_ds[n_name].oindex[ :, :, field_idx, vl_idx] ]

        # filter data that is not part of the desired period
        years_input_file = np.asarray([str(year) for year in self.era5_times[self.range_itime[0]:self.range_itime[1]].astype('datetime64[Y]')])
        valid_years_str = [str(year) for year in range(years[0], (years[1] if len(years)>1 else years[0]) + 1)]
        mask_year = np.isin(years_input_file, valid_years_str)
          
        #self.valid_time_indices = np.where(logical_array)[0]
        self.anchor_itime = np.arange(*self.range_itime)[mask_year]
        self.num_samples = min(self.num_samples, self.anchor_itime.shape[0])
            

    def shuffle(self):

        worker_info = torch.utils.data.get_worker_info()
        rng_seed = None

        if worker_info is not None:
            rng_seed = int(time.time()) // (worker_info.id+1) + worker_info.id

        rng = np.random.default_rng(rng_seed)

        # get random time index
        self.idx_perm_t = rng.choice(self.anchor_itime, self.num_samples // self.batch_size, replace=False)

        idx_lat, idx_lon = rng.choice(self.anchor_ilat, self.num_samples, replace=False), \
                           rng.choice(self.anchor_ilon, self.num_samples, replace=False)

        self.idx_perm_s = np.stack([idx_lat, idx_lon], axis=1)        

    def __iter__(self):
        
        if self.with_shuffle:
            self.shuffle()

        lats, lons = self.era5_lats, self.era5_lons
        n_size = self.n_size
        ts = 1                             # To-Do: add as parsing parameter!!!

        iter_start , iter_end = self.worker_workset()

        for bidx in range( iter_start, iter_end):

            sources, token_infos = [[] for _ in self.input_fields], [[] for _ in self.input_fields]
            targets, target_token_infos = [[] for _ in self.target_fields], [[] for _ in self.target_fields],
            sources_infos, source_idxs = [], []
            target_infos, target_idxs = [], []

            # get matching time index for IMERG dataset
            i_bidx_era5 = self.idx_perm_t[bidx]
            i_bidx_imerg = np.where(self.era5_times[i_bidx_era5] == self.imerg_times)[0][0]

            # get list of time indices for slicing
            idxs_t_era5 = list(np.arange( i_bidx_era5 - n_size[0]*ts, i_bidx_era5, ts, dtype=np.int64))
            idxs_t_imerg = list(np.arange( i_bidx_imerg - n_size[0]*ts, i_bidx_imerg, ts, dtype=np.int64))

            data_t = self.era5_ds['time'][idxs_t_era5[0]:idxs_t_era5[1]].astype(datetime)
            
            # extract data for time steps at hand
            data_era5_tt_sfc = self.era5_ds['data_sfc'][idxs_t_era5]
            data_era5_tt = self.era5_ds['data'][idxs_t_era5]

            data_imerg_tt_sfc = self.imerg_ds['data_sfc'][idxs_t_imerg]

            for sidx in range(self.batch_size):

                idx_era5 = self.idx_perm_s[bidx*self.batch_size+sidx]

                print("Sampled spatial indices:")
                print(idx_era5)
                
                ilat_range_era5 = np.arange(idx_era5[0] - self.nsize_lat[0], idx_era5[0] + self.nsize_lat[1] + 1)
                # handle periodicity in zonal (longitude) direction
                ilon_range_era5 = np.arange(idx_era5[1] - self.nsize_lon[0], idx_era5[1] + self.nsize_lon[1] + 1)
                
                ilon_range_era5 = np.where(ilon_range_era5 < 0, ilon_range_era5 + self.era5_num_lons, ilon_range_era5)
                ilon_range_era5 = ilon_range_era5 % self.era5_num_lons

                # get corresponding spatial sampling indices for IMERG dataset
                ilat_imerg0 = np.where(np.isclose(self.era5_lats[ilat_range_era5[0]] - self.dx_shift[0], self.imerg_lats))[0]
                ilon_imerg0 = np.where(np.isclose(self.era5_lons[ilon_range_era5[0]] - self.dx_shift[1], self.imerg_lons))[0]

                assert len(ilat_imerg0) == 1, f"Could not find required IMERG latitude grid point for first ERA5 grid point at {self.era5_lats[ilat_range_era5[0]]} deg"
                assert len(ilon_imerg0) == 1, f"Could not find required IMERG longitude grid point for first ERA5 grid point at {self.era5_lons[ilon_range_era5[0]]} deg"

                ilat_imerg0, ilon_imerg0 = ilat_imerg0[0], ilon_imerg0[0]

                ilat_range_imerg = np.arange(ilat_imerg0, ilat_imerg0 + self.n_size[1]*self.downscaling_ratio)
                ilon_range_imerg = np.arange(ilon_imerg0, ilon_imerg0 + self.n_size[2]*self.downscaling_ratio)

                # start data retrieval        
                sources_infos += [ [ self.era5_ds['time'][ idxs_t_era5 ].astype(datetime), 
                                     self.era5_lats[ilat_range_era5], self.era5_lons[ilon_range_era5], self.era5_res ] ]
                target_infos += [ [ self.imerg_ds['time'][ idxs_t_imerg ].astype(datetime), 
                                     self.imerg_lats[ilat_range_imerg], self.imerg_lons[ilon_range_imerg], self.imerg_res ] ]
        
                if self.with_source_idxs :
                   source_idxs += [ (idxs_t_era5, ilat_range_era5, ilon_range_era5) ]

                if self.with_target_idxs:
                   target_idxs += [ (idxs_t_imerg, ilat_range_imerg, ilon_range_imerg) ]
        
                # extract input ERA5 data
                for ifield, field_info in enumerate(self.input_fields):  
                  source_lvl, tok_info_lvl  = [], []
                  tok_size  = field_info[4]
                  num_tokens = field_info[3]
                  corr_type = 'global' if len(field_info) <= 6 else field_info[6]
                
                  for ilevel, vl in enumerate(field_info[2]):
                    if vl == 0 : #surface level
                      field_idx = self.era5_ds.attrs['fields_sfc'].index( field_info[0])
                      data_era5_t = data_era5_tt_sfc[ :, field_idx ]
                    else :
                      field_idx = self.era5_ds.attrs['fields'].index( field_info[0])
                      vl_idx = self.era5_ds.attrs['levels'].index(vl)
                      data_era5_t = data_era5_tt[ :, field_idx, vl_idx ]
                  
                    source_data, tok_info = [], []
                    # extract data, normalize and tokenize
                    cdata = data_era5_t[ ... , ilat_range_era5[:, np.newaxis], ilon_range_era5[np.newaxis, :]]

                    # NOTE: uncomment/activate the following after testing
                    normalizer = self.input_normalizers[ifield][ilevel]
                    #if False: #corr_type != 'global': 
                    if corr_type != 'global': 
                      if ilat_range_era5[0] < ilat_range_era5[-1] and ilon_range_era5[0] < ilon_range_era5[-1]:
                        lat_max, lat_min = max(ilat_range_era5), min(ilat_range_era5)
                        lon_max, lon_min = max(ilon_range_era5), min(ilon_range_era5)
                        normalizer = normalizer[:,:,lat_min:lat_max+1,lon_min:lon_max+1]
                      else:
                        normalizer = normalizer[ ... , ilat_range_era5[:,np.newaxis], ilon_range_era5[np.newaxis,:]]
 
                    cdata = normalize(cdata, normalizer, sources_infos[-1][0], year_base = self.year_base)
                    
                    source_data = tokenize( torch.from_numpy( cdata), tok_size )  
                    
                    # NOTE: Remove this after testing
                    #source_data = cdata
                    # token_infos uses center of the token: *last* datetime and center in space
                    dates = self.era5_ds['time'][ idxs_t_era5 ].astype(datetime)
                    cdates = dates[tok_size[0]-1::tok_size[0]]
                    # use -1 is to start days from 0
                    dates = [(d.year, d.timetuple().tm_yday-1, d.hour) for d in cdates] 
                    lats_sidx = self.era5_lats[ilat_range_era5][ tok_size[1]//2 :: tok_size[1] ]
                    lons_sidx = self.era5_lons[ilon_range_era5][ tok_size[2]//2 :: tok_size[2] ]
                    # tensor product for token_infos
                    tok_info += [[[[[ year, day, hour, vl, lat, lon, vl, self.era5_res[0]] for lon in lons_sidx]
                                                                                      for lat in lats_sidx]
                                                                          for (year, day, hour) in dates]]
        
                    source_lvl += [source_data]
                    tok_info_lvl += [ torch.tensor(tok_info, dtype=torch.float32).flatten( 1, -2)]
                      
                  sources[ifield] += [ torch.stack(source_lvl, 0) ]
                  token_infos[ifield] += [ torch.stack(tok_info_lvl, 0) ]

                for ifield, field_info in enumerate(self.target_fields):  
                  target_lvl, tok_info_lvl  = [], []
                  target_tok_size  = field_info[4]
                  target_num_tokens = field_info[3]
                  corr_type = 'global' if len(field_info) <= 6 else field_info[6]  
                
                  for ilevel, vl in enumerate(field_info[2]):
                    if vl == 0 : #surface level
                        field_idx = self.imerg_ds.attrs['fields_sfc'].index( field_info[0])
                        data_t_imerg = data_imerg_tt_sfc[:,field_idx]
                    else :
                        raise ValueError(f"No multi-level data available in IMERG-dataset. Please check the target_field-configuration.")
                  
                    target_data, tok_info = [], []
                    # extract data, normalize and tokenize
                    cdata = data_t_imerg[ ... , ilat_range_imerg[:,np.newaxis], ilon_range_imerg[np.newaxis,:]]

                    # NOTE: uncomment the following after testing
                    normalizer = self.target_normalizers[ifield][ilevel]
                    if corr_type != 'global': 
                      if ilat_range_imerg[0] < ilat_range_imerg[-1] and ilon_range_imerg[0] < ilon_range_imerg[-1]:
                        lat_max, lat_min = max(ilat_range_imerg), min(ilat_range_imerg)
                        lon_max, lon_min = max(ilon_range_imerg), min(ilon_range_imerg)
                        normalizer = normalizer[:,:,lat_min:lat_max+1,lon_min:lon_max+1]
                      else:
                        normalizer = normalizer[ ... , ilat_range_imerg[:,np.newaxis], ilon_range_imerg[np.newaxis,:]]
                    cdata = normalize(cdata, normalizer, sources_infos[-1][0], year_base = self.year_base)
                    
                    target_data = tokenize(torch.from_numpy(cdata), target_tok_size ) 

                    # NOTE: Remove this after testing
                    #target_data = cdata 
                      
                    # token_infos uses center of the token: *last* datetime and center in space
                    dates = self.imerg_ds['time'][ idxs_t_imerg ].astype(datetime)
                    cdates = dates[tok_size[0]-1::tok_size[0]]
                    # use -1 is to start days from 0
                    dates = [(d.year, d.timetuple().tm_yday-1, d.hour) for d in cdates] 
                    lats_sidx = self.imerg_lats[ilat_range_imerg][ target_tok_size[1]//2 :: target_tok_size[1] ]
                    lons_sidx = self.imerg_lons[ilon_range_imerg][ target_tok_size[2]//2 :: target_tok_size[2] ]
                    # tensor product for token_infos
                    tok_info += [[[[[ year, day, hour, vl, lat, lon, vl, self.imerg_res[0]] for lon in lons_sidx]
                                                                                            for lat in lats_sidx]
                                                                          for (year, day, hour) in dates]]
        
                    target_lvl += [target_data]
                    tok_info_lvl += [ torch.tensor(tok_info, dtype=torch.float32).flatten( 1, -2)]      
                  targets[ifield] += [ torch.stack(target_lvl, 0) ]
                  target_token_infos[ifield] += [ torch.stack(tok_info_lvl, 0) ]

            sources = [torch.stack(sources_field) for sources_field in sources]
            token_infos = [torch.stack(tis_field) for tis_field in token_infos]

            targets = [torch.stack(targets_field) for targets_field in targets]
            
            yield ((sources, token_infos), (source_idxs, sources_infos), targets, (target_idxs,target_infos))          # (targets, target_token_infos))
  
    def __len__(self):
        return self.num_samples // self.batch_size


    def worker_workset( self) :

        worker_info = torch.utils.data.get_worker_info()

        if worker_info is None: 
            iter_start = 0
            iter_end = self.num_samples
    
        else:  
            # split workload
            per_worker = len(self) // worker_info.num_workers
            worker_id = worker_info.id
            iter_start = int(worker_id * per_worker)
            iter_end = int(iter_start + per_worker)
            if worker_info.id+1 == worker_info.num_workers :
                iter_end = len(self)

        return iter_start, iter_end

In [ ]:
from atmorep.utils.utils import Config

cf = Config()

#cf.model_id = "3kdutwqb" 
cf.fields = [["velocity_u", [1, 2048, ["velocity_v", "temperature", "specific_humidity", "velocity_z"], 0, ["3k6e6p7o", 141]], [96, 105, 114, 123, 137], [12, 6, 12], [3, 9, 9], [0.7, 0.9, 0.2, 0.05], "local"], 
             ["velocity_v", [1, 2048, ["velocity_u", "temperature", "specific_humidity", "velocity_z"], 1, ["brxmevmt", 141]], [96, 105, 114, 123, 137], [12, 6, 12], [3, 9, 9], [0.7, 0.9, 0.2, 0.05], "local"], 
             ["specific_humidity", [1, 2048, ["velocity_u", "velocity_v", "velocity_z", "temperature", "total_precip"], 2, ["ctxc97nr", 128]], [96, 105, 114, 123, 137], [12, 6, 12], [3, 9, 9], [0.85, 0.9, 0.2, 0.05], "local"], 
             ["velocity_z", [1, 1024, ["velocity_u", "velocity_v", "temperature"], 0, ["15oisw8d", 273]], [96, 105, 114, 123, 137], [12, 6, 12], [3, 9, 9], [0.65, 0.9, 0.2, 0.05], "global"],
             ["temperature", [1, 1536, ["velocity_u", "velocity_v", "velocity_z", "specific_humidity"], 3, ["3qou60es", 327]], [96, 105, 114, 123, 137], [12, 2, 4], [3, 27, 27], [0.85, 0.9, 0.2, 0.05], "local"],
             ["total_precip", [1, 1536, ["velocity_u", "velocity_v", "velocity_z", "specific_humidity"], 3, ["3kdutwqb", 900]], [0], [12, 6, 12], [3, 9, 9], [0.5, 0.9, 0.1, 0.05]]]
cf.input_fields = cf.fields

cf.downscaling_ratio = 3
cf.fields_downscaling = [ ['total_precip', 
                        [1,1536,["velocity_u","velocity_v","specific_humidity"]],
                        [0],
                        [12,6,12],
                        [3,9*cf.downscaling_ratio,9*cf.downscaling_ratio], 
                        1.0 ] ]
cf.target_fields = cf.fields_downscaling
cf.input_file_path = "/p/scratch/atmo-rep/data/era5_1deg/months/era5_y1979_2021_res025_chunk8.zarr"
cf.target_file_path = "/p/scratch/atmo-rep/data/imerg/imerg_regridded/imerg_regrid_y2003_2021_res083_chunk8.zarr"
cf.years_train = [2003,2020]
cf.years_val = [2021]  #[2018] 
cf.month = None
cf.batch_size = 8
cf.num_samples_per_epoch = 24
cf.n_size = [36, 9*6, 9*12]

In [ ]:
cf.downscaling_ratio

In [ ]:
downscaling_dataset = MultifieldDownscalingSampler(
                    cf.input_file_path,
                    cf.target_file_path,
                    cf.fields,
                    cf.target_fields,
                    cf.years_val,
                    cf.batch_size,
                    cf.n_size,
                    cf.num_samples_per_epoch,
                    cf.downscaling_ratio,
                    with_shuffle=True
        )

In [ ]:
len(downscaling_dataset)

In [ ]:
iter_ds = iter(downscaling_dataset)

In [ ]:
(sources, token_infos), (source_idxs, sources_infos), targets, (target_idxs,target_infos) = next(iter_ds)

In [ ]:
sample_idx = 0
era5_sample = sources[-1][sample_idx,0]
imerg_sample = targets[0][sample_idx,0]

In [ ]:
def detokenize( data):
    data = data.permute( [*np.arange( len(data.shape)-5), -3, -5, -2, -4, -1])
    data = data.reshape( [*data.shape[:-6], np.prod( data.shape[-6:-4]), # time
                                          np.prod( data.shape[-4:-2]), # lat
                                          np.prod( data.shape[-2:])])  #lon
    return data


In [ ]:
tokenize=True
normalize=True

if tokenize:
    era5_sample = detokenize(era5_sample)
    imerg_sample = detokenize(imerg_sample)

if normalize:
    from atmorep.datasets.normalizer import denormalize
    era5_sample = denormalize(era5_sample, downscaling_dataset.input_normalizers[5][0], sources_infos[sample_idx][0],downscaling_dataset.year_base)
    imerg_sample = denormalize(imerg_sample, downscaling_dataset.target_normalizers[0][0], target_infos[sample_idx][0],downscaling_dataset.year_base)


In [ ]:
###test for temp parameter as normalizer set there is local whereas for precipitation it is global

# era5_temp_sample = sources[-1][sample_idx,0]

# if tokenize:
#     era5_temp_sample = detokenize(era5_temp_sample)

# if normalize:
#     era5_temp_sample = denormalize(era5_temp_sample, downscaling_dataset.input_normalizers[4][0][:,:,source_idxs[sample_idx][1][:,np.newaxis],source_idxs[sample_idx][2][np.newaxis,:]], sources_infos[sample_idx][0], downscaling_dataset.year_base)
    

In [ ]:
precip_era5 = xr.DataArray(era5_sample*1000., dims=["time", "lat", "lon"], 
                           coords={"time": sources_infos[sample_idx][0], "lat": math_to_geolats(sources_infos[sample_idx][1]), "lon": np.where(sources_infos[sample_idx][2] >180.0,
                                                                                                                                               sources_infos[sample_idx][2] - 360.0,
                                                                                                                                               sources_infos[sample_idx][2])})

precip_imerg = xr.DataArray(imerg_sample, dims=["time", "lat", "lon"], 
                           coords={"time": target_infos[sample_idx][0], "lat": math_to_geolats(target_infos[sample_idx][1]),
                                   "lon": np.where(target_infos[sample_idx][2] > 180., target_infos[sample_idx][2] - 360.0, target_infos[sample_idx][2])})

In [ ]:
precip_era5['lat'], precip_imerg['lat']

In [ ]:
precip_era5['lon'], precip_imerg['lon']

In [ ]:
%matplotlib inline

# parameters for plotting
tidx = 9
val_range = {"vmin": 0, "vmax": 2.}

precip_imerg.isel({"time": tidx}).plot(**val_range)

In [ ]:
precip_era5.isel({"time": tidx}).plot(**val_range)

In [ ]:
isample = 3

imerg_sample = (targets[0].numpy())[isample, 0, ...]
era5_sample = (sources[-1].numpy())[isample, 0, ...]

In [ ]:
year, day, hour = tuple(token_infos[-1].numpy()[isample, 0, 0, 0][0:3])

date_start = pd.to_datetime(year * 1000 + day, format='%Y%j')
date_start = date_start + pd.to_timedelta(hour, unit='h') - pd.to_timedelta(1, unit='h')
all_dates = pd.date_range(date_start, periods=36, freq="1h")

# sanity check
year, day, hour = tuple(token_infos[-1].numpy()[isample,0,0,-1][0:3])

date_end = pd.to_datetime(year * 1000 + day, format='%Y%j')
date_end = date_end + pd.to_timedelta(hour, unit='h') + pd.to_timedelta(1, unit='h')

assert all_dates[-1] == date_end, "Inconsistent date information retrieved from token info"

In [ ]:
date_start, date_end

In [ ]:
lat_min, lat_max = token_infos[-1].numpy()[0, 0, 0, 0][[4]], token_infos[-1].numpy()[0, 0, 0, -1][[4]]
lon_min, lon_max = token_infos[-1].numpy()[0, 0, 0, 0][[5]], token_infos[-1].numpy()[0, 0, 0, -1][[5]]

if lon_min > lon_max:
    lon_min -= 360.

lat_range_era5 = [lat_min[0] - 4*0.25, lat_max[0] + 4*.25]
lon_range_era5 = [lon_min[0] - 4*0.25, lon_max[0] + 4*.25]

lat_range_imerg = [lat_range_era5[0] - 0.25/3, lat_range_era5[1] + 0.25/3]
lon_range_imerg = [lon_range_era5[0] - 0.25/3, lon_range_era5[1] + 0.25/3]

In [ ]:
lats_era5, lons_era5 = get_latlon_arr(lat_range_era5, lon_range_era5, .25)
lats_imerg, lons_imerg = get_latlon_arr(lat_range_imerg, lon_range_imerg, .25/3)

In [ ]:
precip_era5 = xr.DataArray(era5_sample*1000., dims=["time", "lat", "lon"], 
                           coords={"time": all_dates, "lat": math_to_geolats(lats_era5), "lon": lons_era5})

precip_imerg = xr.DataArray(imerg_sample, dims=["time", "lat", "lon"], 
                           coords={"time": all_dates, "lat": math_to_geolats(lats_imerg), "lon": lons_imerg})

In [ ]:
%matplotlib inline

# parameters for plotting
tidx = 7
val_range = {"vmin": 0, "vmax": 2.}

precip_imerg.isel({"time": tidx}).plot(**val_range)

In [ ]:
precip_era5.isel({"time": tidx}).plot(**val_range)

In [ ]:
from pathlib import Path
import xarray as xr
import matplotlib.pyplot as plt

datadir = Path("/p/project1/deepacf/atmo-rep/patnala1/nc_files_new")
ds_imerg = xr.open_dataset(datadir.joinpath("imerg_25722_44_78.nc"))
ds_era5 = xr.open_dataset(datadir.joinpath("era_5_25722_109_2.nc"))

precip_era5 = ds_era5["total_precip"]
precip_imerg = ds_imerg["total_precip"]

In [ ]:
%matplotlib inline

In [ ]:
tidx = 0

ds_imerg["total_precip"].isel({"time": tidx}).plot(vmin=0., vmax=2.)

plt.show()

In [ ]:
ds_era5["lons"] = ds_era5["lons"].where(ds_era5["lons"] < 180., ds_era5["lons"] - 360.)

In [ ]:
ds_era5["total_precip"].isel({"time": tidx}).plot(vmin=0., vmax=2.)

# Plot data from raw datafiles for sanity check

In [ ]:
datadir_imerg_raw = Path("/p/scratch/atmo-rep/data/imerg/imerg_regridded")
datadir_era5_raw = Path("/p/scratch/atmo-rep/data/era5/new_structure/total_precip/ml0")

date_now = pd.to_datetime(precip_era5["time"][tidx].values)
ym_str = date_now.strftime("y%Y_m%m")

fname_imerg_raw = datadir_imerg_raw.joinpath(f"3B-HHR.MS.MRG.3IMERG.{ym_str}.nc")
fname_era5_raw = datadir_era5_raw.joinpath(f"era5_total_precip_{ym_str}_ml0.nc")

In [ ]:
print(f"Read raw IMERG-data from {fname_imerg_raw}")
ds_imerg_raw = xr.open_dataset(fname_imerg_raw)

print(f"Read raw ERA5-data from {fname_era5_raw}")
ds_era5_raw = xr.open_dataset(fname_era5_raw)

In [ ]:
offset = 0

precip_imerg_raw = ds_imerg_raw["precipitation"].sel({"lat": precip_imerg['lat'], "lon": precip_imerg['lon'], 
                                                     "time": date_now + pd.Timedelta(offset,"h")}, method='nearest')

precip_era5_raw = ds_era5_raw["tp"].sel({"latitude": precip_era5['lat'], "longitude": precip_era5['lon'], 
                                         "time": date_now + pd.Timedelta(offset,"h")})*1000.

In [ ]:
import matplotlib.pyplot as plt
precip_imerg_raw.plot(vmin=0., vmax=2.)

plt.show()

In [ ]:
precip_era5_raw.plot(vmin=0., vmax=2.)

plt.show()

In [ ]:
assert np.amax(np.abs(precip_era5_raw - precip_era5.isel({"time": tidx}))) < 5.e-04, "ERA5-data from file and from torch dataset are different."
assert np.amax(np.abs(precip_imerg_raw - precip_imerg.isel({"time": tidx}))) < 1.e-06, "IMERG-data from file and from torch dataset are different."